# TEO - Gerador de Instâncias

In [407]:
import pandas as pd
import json
import csv
import sympy
import random

## Modelagem das estruturas (até agora)

### Eventos

Eventos são efetivamente as matérias que serão incluídas nos timeslots. Cada evento possui um código, o nome, o período associado (se a matéria é obrigatória) e a lista de pré-requisitos daquele evento.\
Dentro do vetores de alunos, haverá a probabilidade de o aluno não ter conseguido concluir uma matéria num período anterior, de modo que caso isso seja verdade, a matéria deverá aparecer como opcional para ele na função objetivo, pois haverá a possibilidade do indivíduo fazê-lo se tiver oportunidade para se reperiodizar. Além disso, qualquer pré-requisito não concluído indica que a matéria é trancada por ela deve ser retirada das disciplinas obrigatórias.\
Além disso, após a geração dos estudantes, uma lista de prioridades de optativas e mandatórias para alunos que repetiram deverá ser criada para a lista de disciplinas selecionadas.

In [7]:
df_events = pd.read_csv("/home/leandrobataglia/Documentos/SA-Timetabling/TEO-events.csv")
df_events.head()

,cod_event,event_name,event_period,event_type,cod_prerequisites
0,IM853,CIRCUITOS DIGITAIS,1.0,MANDATORY,NaN
1,IM885,GEOMETRIA ANALÍTICA,1.0,MANDATORY,NaN
2,TN703,COMPUTADORES E SOCIEDADE,1.0,MANDATORY,NaN
3,TN705,MATEMÁTICA DISCRETA PARA COMPUTAÇÃO,1.0,MANDATORY,NaN
4,TN706,PROGRAMAÇÃO ESTRUTURADA,1.0,MANDATORY,NaN


### Professores

A lista de professores é utilizada, principalmente, para gerar a matriz de conflito em relação às disciplinas selecionadas.\
Além disso, referencia as preferências dos horários dos professores para minimização. Não será tentada na primeira formulação do problema.

In [6]:
df_professors = pd.read_csv("/home/leandrobataglia/Documentos/SA-Timetabling/TEO-professors.csv")
df_professors.head()

,cod_professor,professor_name,cod_preferred_hours
0,PROF_1,ADRIELI FLAUTA,"TER_8-10, TER_10-12, TER_14-16, TER_16-18"
1,PROF_2,BRENO DEMBOFEXILE,"SEX_8-10, SEX_10-12, SEX_14-16, SEX_16-18"
2,PROF_3,DANILO POSER,NaN
3,PROF_4,PKMN-GO,"SEG_8-10, SEG_10-12, SEG_14-16, SEG_16-18, TER..."
4,PROF_5,GRAPH QUEEN,NaN


### Estudantes

Os estudantes são instâncias artificiais geradas a partir de um conjunto de demandas dos alunos para um dado semestre. Para tal, a cada instância, são consideradas apenas as mandatórias (período par ou ímpar) para as quais os estudantes são elegíveis, e uma função de probabilidade é rodada para definir se um determinado aluno "deve" uma matéria mandatória, e quantas matérias serão escolhidas pelo aluno da lista de optativas, caso não haja conflito com as mandatórias. Cada conflito de optativas incorrerá em uma penalidade na função objetivo, cuja importância varia de acordo com o período (quanto mais maior o peso).\
Além disso, para cada optativa para usuários dos períodos 6 em diante, será entendido como mandatórias, de acordo com o que está definido quanto à quantidade de optativas os alunos deste período precisam fazer.

### Timeslots

São os espaços disponíveis de modo a preencher a lista de 

In [8]:
df_timeslots = pd.read_csv("/home/leandrobataglia/Documentos/SA-Timetabling/TEO-timeslots.csv")
df_timeslots.head()

,cod_timewindow,time group_1,time group_2
0,SEG_8-10,True,False
1,SEG_10-12,True,True
2,SEG_14-16,True,True
3,SEG_16-18,False,True
4,TER_8-10,True,False


In [442]:
class Student():
    
    def __init__(self, name, semester, cod_assigned_mandatory_events, cod_pending_mandatory_events):
        self.name = name
        self.semester = semester
        self.cod_assigned_mandatory_events = cod_assigned_mandatory_events
        self.cod_pending_mandatory_events = cod_pending_mandatory_events
        self.cod_chosen_optative_events = []

In [535]:
class Event:
    def __init__(self, cod_event, event_name, event_period, event_type, cod_timegroup, cod_prerequisites):
        self.cod_event = cod_event
        self.event_name = event_name
        self.event_period = event_period
        self.event_type = event_type
        self.cod_timegroup = cod_timegroup
        self.cod_prerequisites = cod_prerequisites
        self.student_pending_counter = 0
        
        pass
    pass

In [551]:
class Course:
    
    path = "/home/leandrobataglia/Documentos/SA-Timetabling/"
    instances = "instances/"
            
    def __init__(self):
        
        self.students = [] #recebe as instâncias de estudantes
        self.mandatory_events = []
        self.optative_events = []
        
        
        
    def load_events(self, number_optionals):
        f_mandatory = open(type(self).path + "TEO-events_mandatory.json")
        f_optional = open(type(self).path + "TEO-events_optional.json")
        
        data = json.load(f_mandatory)
        for item in data:
            event_object =  Event(item["cod_event"],
                                item["event_name"], 
                                item["event_period"],
                                item["event_type"],
                                item["cod_timegroup"],
                                item["cod_prerequisites"])
            self.mandatory_events.append(event_object)
                
                
        data = json.load(f_optional)
        for i in range(number_optionals):
            event_index = random.randint(0, len(data)-1) 
            
            event_object =  Event(data[event_index]["cod_event"],
                                data[event_index]["event_name"], 
                                data[event_index]["event_period"],
                                data[event_index]["event_type"],
                                data[event_index]["cod_timegroup"],
                                data[event_index]["cod_prerequisites"])
            self.optative_events.append(event_object)
            data.remove(data[event_index])
            
            
    
    # UM MÉTODO PARA GERAR AS INSTÂNCIAS DE  ESTUDANTES, DEVIDAMENTE TRATADAS 
    # PARA QUE NÃO INCLUA PRÉ-REQUISITOS.PARA ISSO,ALÉM DISSO, SALVAR NUM CSV
    # PARA QUE POSSA SERVIR COMO O CONJUNTO DE ALUNOS DE UMA INSTÂNCIA DO PROBLEMA.
    def generate_students(self, total_students, year_students, retention_odds, semester):
        
        def pick_classes(student_period, retention_odds):
            assigned_mandatory = []
            pending_mandatory = []
            chosen_optatives = []
            optatives_list = self.optative_events
            
            #PROCEDIMENTO DAS OBRIGATÓRIAS
            for event in self.mandatory_events:
                for i in range(2):
                        if event.event_period != 1:
                            if event.event_period == student_period - i:
                                if (random.random() < retention_odds):
                                    pending_mandatory.append(event.cod_event)
                        
            for event in self.mandatory_events:           #para cada evento mandatório
                if event.event_period == student_period:  #se o estudante for daquele período
                    if event.cod_prerequisites != None:   #se pré-requisitos não estiver vazia
                        ##print("cod_prerequisites não-vazia")
                        flag = False
                        for x in event.cod_prerequisites: #para cada evento em pré-requisitos
                            for y in pending_mandatory:
                                if x == y:
                                    flag = True
                        if flag == True:
                            continue
                        else:
 
                            assigned_mandatory.append(event.cod_event)
                    else:
                        assigned_mandatory.append(event.cod_event)
                 
            # LIMPA DUPLICATAS
            pending_mandatory = list(dict.fromkeys(pending_mandatory))
            
#             # PROCEDIMENTO DAS OPTATIVAS
#             # Se o aluno não tem pendências, tem 5% de chance de optar por fazer alguma optativa
#             if student_period > 1 and student_period < 6 and pending_mandatory == None and random.random() < 0.05:
#                 optative_random_index = random.randint(0, len(self.optative_events)-1)
#                 chosen_optatives.append(self.optative_events[optative_random_index].cod_event) 
            
            #Concluintes podem selecionar até 4 optativas 
            if student_period >= 6:
                for i in range(4):
                    
                    # Concluintes tem uma chance de dever alguma obrigatória do período anterior
                    if random.random() < 0.06:
                        mandatory_index = random.randint(0, len(self.mandatory_events)-1)
                        pending_mandatory.append(self.mandatory_events[mandatory_index].cod_event)
            
            return assigned_mandatory, pending_mandatory
        
        
        counter = 1
        if semester == "odd":
            student_period = 1
        elif semester == "even":
            student_period = 2
            
        for students in year_students:
            total_students = total_students - students
            
            for student in range(students):
                assigned_mandatory, pending_mandatory =  pick_classes(student_period, retention_odds)
                self.students.append(Student(counter, student_period, assigned_mandatory, pending_mandatory))
                counter = counter + 1
            
            student_period = student_period + 2
            
        for student in range(total_students):
            assigned_mandatory, pending_mandatory =  pick_classes(student_period, retention_odds)
            self.students.append(Student(counter, student_period, assigned_mandatory, pending_mandatory))
            counter = counter + 1
        pass
    
    def save_instances(self, filename, students_per_year, semester):
        students_list = []
        events_list = []
        
        if semester == "odd":
            event_period = 1
        elif semester == "even":
            event_period = 2
            
        def save_students(self, filename):
            
            students_list = []
            
            for student in self.students:
                
                students_list.append({"name":student.name,
                                      "semester":student.semester,
                                      "cod_assigned_mandatory_events":student.cod_assigned_mandatory_events,
                                      "cod_pending_mandatory_events":student.cod_pending_mandatory_events,
                                      "cod_chosen_optative_events":student.cod_chosen_optative_events})
                
                with open(self.path + self.instances + filename + "_students" + ".json", "w") as json_file:
                    
                    json_string = json.dumps(students_list)
                    json_file.write(json_string)
                                        
        
        save_students(self, filename)
        
        event_p = event_period
        dict_students_per_year = dict()
        for year in students_per_year:
            dict_students_per_year.update({event_p: year})
            event_p += 2
        
        for student in self.students:
            for event in self.mandatory_events:
                for item in student.cod_pending_mandatory_events:
                    if item == event.cod_event:
                        event.student_pending_counter += 1
                    
                        
        for event in self.mandatory_events:
            if event_period == event.event_period or event.student_pending_counter:
                events_list.append(event)
                
                
        events_list.extend(self.optative_events)
        
        csvfile = open(self.path + self.instances + filename + ".csv", "w")
        writer = csv.writer(csvfile)
        
        writer.writerow(["cod_event", 
                         "event_name", 
                         "event_period", 
                         "event_type", 
                         "students_assigned", 
                         "student_pending_counter",
                         "assigned_professor"])
        
        for event in events_list:
            
            if event.event_type == "MANDATORY" and ((event.event_period - event_period) % 2) == 0:
                
                writer.writerow([event.cod_event, 
                                 event.event_name, 
                                 event.event_period, 
                                 event.event_type, 
                                 dict_students_per_year[event.event_period],
                                 event.student_pending_counter,
                                 event.cod_timegroup])
                
            elif event.event_type == "MANDATORY" and ((event.event_period - event_period) % 2) != 0:
                
                writer.writerow([event.cod_event, 
                                 event.event_name, 
                                 event.event_period, 
                                 "PENDING", 
                                 0, 
                                 event.student_pending_counter,
                                 event.cod_timegroup])
            else:
                
                writer.writerow([event.cod_event, 
                                 event.event_name, 
                                 event.event_period, 
                                 event.event_type, 
                                 0, 
                                 event.student_pending_counter,
                                 event.cod_timegroup])
        csvfile.close()
           
    def generate_event_json(self, filename):
        
        events_list = []
        
        with open(self.path + self.instances + filename + ".csv", "r") as file:
        
            for row_no, row in enumerate(csv.reader(file)):
                
                if row_no == 0:
                    print("Headers: ", row)
                else:
                    events_list.append({"cod_event":row[0], 
                                        "event_name":row[1], 
                                        "event_period":int(row[2]), 
                                        "event_type":row[3], 
                                        "students_assigned":int(row[4]), 
                                        "student_pending_counter":int(row[5]),
                                        "cod_timegroup":int(row[6]),
                                        "assigned_professor":row[7]})
                    

        json_string = json.dumps(events_list)
        json_file = open(self.path + self.instances + filename + "_events" +".json", "w")
        json_file.write(json_string)
        json_file.close()
        
    @classmethod
    def generate_event_json(cls, filename):
        
        events_list = []
        
        with open(cls.path + cls.instances + filename + ".csv", "r") as file:
        
            for row_no, row in enumerate(csv.reader(file)):
                
                if row_no == 0:
                    print("Headers: ", row)
                else:
                    events_list.append({"cod_event":row[0], 
                                        "event_name":row[1], 
                                        "event_period":int(row[2]), 
                                        "event_type":row[3], 
                                        "students_assigned":int(row[4]), 
                                        "student_pending_counter":int(row[5]), 
                                        "cod_timegroup":row[6],
                                        "assigned_professor":int(row[7])})
                    

        json_string = json.dumps(events_list)
        json_file = open(self.path + self.instances + filename + "_events" +".json", "w")
        json_file.write(json_string)
        json_file.close()

In [553]:
course = Course()
course.load_events(number_optionals = 14)
for item in course.optative_events:
    print(item.event_name)

APRENDIZADO DE MÁQUINA
TEORIA DOS GRAFOS
ALGORITMOS PARALELOS E DISTRIBUÍDOS
TÓPICOS ESPECIAIS EM OTIMIZAÇÃO
INTELIGÊNCIA DE NEGÓCIOS
ENGENHARIA DE REQUISITOS
EMPREENDEDORISMO EM INFORMÁTICA
INTRODUÇÃO À BIOLOGIA COMPUTACIONAL
GERÊNCIA DE PROJETOS
TÓPICOS ESPECIAIS EM GRAFOS E ALGORITMOS
OTIMIZAÇÃO COMBINATÓRIA
MODELAGEM E ANÁLISE FORMAL DE PROCESSOS DE NEGÓCIO
TEORIA DOS JOGOS ALGORÍTMICA
REDES DE COMPUTADORES SEM FIO


In [530]:
course.generate_students(185, [60,45,30,20], 0.1, "even")

In [531]:
## CUIDADO!!!!! TROQUE O NOME PARA NÃO APAGAR INSTÂNCIAS ANTERIORES

instance_name = "test"

In [532]:
course.save_instances(instance_name, [60,45,30,20], "even")

In [ ]:
course.generate_event_json(instance_name)